In [1]:
    #Import modules and data csv for analysis
import pandas as pd
import numpy as np

file_to_load = "purchase_data.csv"

purchase_data = pd.read_csv(file_to_load)
    #print out .head to see what information is in data csv
purchase_data.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


In [2]:
    #Find number of unique players as well as gender
player_count = purchase_data.loc[:, ["SN", "Age", "Gender"]]
    #Filter out players with more than one purchase
player_count = player_count.drop_duplicates()
    #Place unique players in list for later use
total_players = player_count.count()[0]

pd.DataFrame({"Total Players": [total_players]})

,Total Players
0,576


In [3]:
    #Finding number of unique items
unique_items = len(purchase_data["Item ID"].unique())
    #Finding average purchase price
avg_price = purchase_data["Price"].mean()
    #Finding total number of purchases
number_of_purchases = purchase_data["Price"].count()
    #Finding total revenue
total_value = purchase_data["Price"].sum()
    #Putting Values in Data Frame
revenue_df = pd.DataFrame({"Number of Unique Items": unique_items, "Average Price": [avg_price],
                           "Number of Purchases": [number_of_purchases], "Total Revenue": [total_value]})
    #Formating tables for place values
revenue_df["Average Price"] = revenue_df["Average Price"].map("${:.2f}".format)
revenue_df["Total Revenue"] = revenue_df["Total Revenue"].map("${:,.2f}".format)

revenue_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [4]:
    #Gender Demographics
gender_demo = player_count["Gender"].value_counts()
    #Percentage of players per "Gender" group
gender_demo_percent = gender_demo / total_players*100
    #Putting Values in Data Frame
gender_demo_df = pd.DataFrame({"Total Count": gender_demo, "Percentage of Players": gender_demo_percent})
    #Formating table for place values
gender_demo_df["Percentage of Players"] = gender_demo_df["Percentage of Players"].map("{:.2f}%".format)
gender_demo_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [5]:
    #Gender purchase breakdown by category
purchase_count = purchase_data.groupby(["Gender"]).count()["Price"].rename("Purchase Count")
avg_purchase_price = purchase_data.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
total_purchase_value = purchase_data.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_avg_purchase = total_purchase_value / gender_demo_df["Total Count"]
    #Putting Values into Data Frame
purchasing_analysis_df = pd.DataFrame({"Purchase Count": purchase_count, 
                                       "Average Purchase Price": avg_purchase_price,
                                       "Total Purchase Value": total_purchase_value, 
                                       "Average Total Per Person": gender_avg_purchase})
    #Formating table
purchasing_analysis_df["Average Purchase Price"] = avg_purchase_price.map("${:,.2f}".format)
purchasing_analysis_df["Total Purchase Value"] = total_purchase_value.map("${:,.2f}".format)
purchasing_analysis_df["Average Total Per Person"] = gender_avg_purchase.map("${:,.2f}".format)
purchasing_analysis_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [6]:
    #Age breakdown

    #Create bins for age and age groups
bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 9999]
age_groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

    #Putting players into bins by age group
player_count["Age Groups"] = pd.cut(player_count["Age"], bins, labels=age_groups)
    #Count of each age range
age_totals = player_count["Age Groups"].value_counts()
    #Percent of each age group
age_percents = age_totals / total_players
    #Putting Values into Data Frame
age_breakdown = pd.DataFrame({"Total Count": age_totals, 
                              "Percentage of Players": age_percents})
    #formating table
age_breakdown['Percentage of Players'] = age_percents.map("{:,.2%}".format)
    #sorting index so that index starts at <10 
age_breakdown = age_breakdown.sort_index()

age_breakdown


,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [7]:
    #Creating bins
purchase_data["Age groups"] = pd.cut(purchase_data["Age"], bins, labels=age_groups)

    #Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc.
age_purchase_counts = purchase_data.groupby(["Age groups"]).count()["Price"].rename("Purchase Count")
age_average_price = purchase_data.groupby(["Age groups"]).mean()["Price"].rename("Average Purchase Price")
age_average_total = purchase_data.groupby(["Age groups"]).sum()["Price"].rename("Total Purchase Value")
age_avg_per = age_average_total / age_breakdown["Total Count"]
    
    #Putting Values into Data Frame
purchase_analysis_age = pd.DataFrame({"Purchase Count": age_purchase_counts, 
                         "Average Purchase Price": age_average_price, 
                         "Total Purchase Value": age_average_total, 
                         "Avg Total Purchase per Person": age_avg_per})

    #Formating table 
purchase_analysis_age["Average Purchase Price"] = age_average_price.map("${:,.2f}".format)
purchase_analysis_age["Total Purchase Value"] = age_average_total.map("${:,.2f}".format)
purchase_analysis_age["Avg Total Purchase per Person"] = age_avg_per.map("${:,.2f}".format)
purchase_analysis_age = purchase_analysis_age.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]
purchase_analysis_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [8]:
    #Top Spenders
    #Run basic calculations to obtain the results for the table below
    # of unique spenders that bought items
spender_count = purchase_data.groupby(["SN"]).count()["Price"].rename("Purchase Count")
    #Average of purchases per spender
spender_avg = purchase_data.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
    #Total of Purchsaes per Person
spender_total = purchase_data.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
    #Putting Values into Data Frame
spender_data = pd.DataFrame({"Purchase Count": spender_count, 
                          "Average Purchase Price": spender_avg,
                          "Total Purchase Value": spender_total})
    #Sorting Data Frame for person who spent the most on items to be at the top of the list
spender_sort = spender_data.sort_values("Total Purchase Value", ascending=False)
    #Formating table
spender_sort["Average Purchase Price"] = spender_avg.map("${:,.2f}".format)
spender_sort["Total Purchase Value"] = spender_total.map("${:,.2f}".format)
spender_sort = spender_sort.loc[:,["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

spender_sort.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [9]:
    #Most Popular Items
    
item_name_data = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]
    #Number of each item purchased
item_purchase_count = item_name_data.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")
    #Item price
total_purchase_value = item_name_data.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
    #Average of money spent per item
average_item_price = item_name_data.groupby(["Item ID", "Item Name"]).mean()["Price"]

    #Putting Values into Data Frame
items_data_pd = pd.DataFrame({"Purchase Count": item_purchase_count, 
                             "Item Price": average_item_price, 
                             "Total Purchase Value": total_purchase_value}) 
    #Sorting Data Frame for highest value item to be top of the list                              
items_sorted_pd = items_data_pd.sort_values("Purchase Count", ascending=False)
    #Formating Table
items_sorted_pd["Item Price"] = average_item_price.map("${:,.2f}".format)
items_sorted_pd["Total Purchase Value"] = total_purchase_value.map("${:,.2f}".format)
items_sorted_pd.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [10]:
    #Most profitable items sorted with most profitable first
items_profit_total = items_data_pd.sort_values("Total Purchase Value", ascending=False)

    #Formating Table
items_profit_total["Purchase Count"] = items_profit_total["Purchase Count"].map("{:,}".format)   
items_profit_total["Item Price"] = items_profit_total["Item Price"].map("${:,.2f}".format)
items_profit_total["Total Purchase Value"] = items_profit_total["Total Purchase Value"].map("${:,.2f}".format)
     #Putting Values into Data Frame
most_profitable = items_profit_total.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]
most_profitable.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
